# AD Knowledge Graph 

In [ ]:
import numpy as np
import pandas as pd
import time
import re
import math
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn import metrics 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable
from torch.nn.modules import Module
from torch.utils.data import Dataset, DataLoader

from torch_geometric.data import Data
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils import add_remaining_self_loops, add_self_loops
from torch_geometric.utils import to_undirected
from torch_geometric.nn import GCNConv, GCN2Conv, SAGEConv,GAE, VGAE
from torch.nn import Linear

In [ ]:
#data_path='./'
#exp_id='v0'
device='cuda:4'
device_id=4 #'cpu' if CPU, device number if GPU
embedding_size=128

In [ ]:
data_path = './AD_project/pre_process_data/'
exp_id='exp1'
le=pickle.load(open(data_path+'AD_LabelEncoder_'+exp_id+'_kh.pkl', 'rb'))
edge_index=pickle.load(open(data_path+'AD_edge_index_'+exp_id+'_kh.pkl','rb'))
edge_index=edge_index[edge_index.type!='AD_related_inf']
node_feature_np=pickle.load(open(data_path+'AD_node_feature_'+exp_id+'_kh.pkl','rb'))
node_class=pickle.load(open(data_path+'AD_gene_node_class_'+exp_id+'_kh.pkl','rb'))
node_class_mask=pickle.load(open(data_path+'gene_mask_'+exp_id+'_kh.pkl','rb'))
AD_gene_index=pickle.load(open(data_path+'ad_gene_index_'+exp_id+'_kh.pkl','rb'))
non_AD_gene_index=pickle.load(open(data_path+'non_ad_gene_index_'+exp_id+'_kh.pkl','rb'))

In [ ]:
train_AD_gene_node,test_AD_gene_node=train_test_split(AD_gene_index,test_size=0.1)
train_non_AD_gene_node,test_non_AD_gene_node=train_test_split(AD_gene_index,test_size=0.1)
train_node_index=train_AD_gene_node+train_non_AD_gene_node
test_node_index=test_AD_gene_node+test_non_AD_gene_node
train_node_edge_index=edge_index[(edge_index.node1.isin(train_node_index))|(edge_index.node2.isin(train_node_index))]
test_node_edge_index=edge_index[(edge_index.node1.isin(test_node_index))|(edge_index.node2.isin(test_node_index))]
all_node_edge_index=pd.concat([train_node_edge_index,test_node_edge_index])

In [ ]:
node_feature=torch.tensor(node_feature_np, dtype=torch.float)

In [ ]:
edge_attr_dict={'gene-drug':0,'protein-protein':1, 'drug-phenotype':2,'gene-phenotype':3,'drug_pathway':4,'drug_sim':5,'AD_gene_pathway':6}#'mutation':3,

edge_index['type']=edge_index['type'].apply(lambda x: edge_attr_dict[x])

## Batch

In [ ]:
def train_test_split_edges(data, val_ratio=0.05, test_ratio=0.1):
    r"""Splits the edges of a :obj:`torch_geometric.data.Data` object
    into positive and negative train/val/test edges, and adds attributes of
    `train_pos_edge_index`, `train_neg_adj_mask`, `val_pos_edge_index`,
    `val_neg_edge_index`, `test_pos_edge_index`, and `test_neg_edge_index`
    to :attr:`data`.

    Args:
        data (Data): The data object.
        val_ratio (float, optional): The ratio of positive validation
            edges. (default: :obj:`0.05`)
        test_ratio (float, optional): The ratio of positive test
            edges. (default: :obj:`0.1`)

    :rtype: :class:`torch_geometric.data.Data`
    """

    assert 'batch' not in data  # No batch-mode.

    num_nodes = data.num_nodes
    row, col = data.edge_index
    #data.edge_index = None
    attr = data.edge_attr

    # Return upper triangular portion.
    #mask = row < col
    #row, col = row[mask], col[mask]

    n_v = int(math.floor(val_ratio * row.size(0)))
    n_t = int(math.floor(test_ratio * row.size(0)))

    # Positive edges.
    perm = torch.randperm(row.size(0))
    row, col = row[perm], col[perm]
    attr=attr[perm]

    r, c = row[:n_v], col[:n_v]
    data.val_pos_edge_index = torch.stack([r, c], dim=0)
    data.val_pos_edge_attr = attr[:n_v]
    
    r, c = row[n_v:n_v + n_t], col[n_v:n_v + n_t]
    data.test_pos_edge_index = torch.stack([r, c], dim=0)
    data.test_post_edge_attr = attr[n_v:n_v + n_t]

    r, c = row[n_v + n_t:], col[n_v + n_t:]
    data.train_pos_edge_index = torch.stack([r, c], dim=0)
    data.train_pos_edge_attr = attr[n_v+n_t:]

    # Negative edges.
    neg_adj_mask = torch.ones(num_nodes, num_nodes, dtype=torch.uint8)
    neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
    neg_adj_mask[row, col] = 0

    neg_row, neg_col = neg_adj_mask.nonzero().t()
    perm = random.sample(range(neg_row.size(0)),
                         min(n_v + n_t, neg_row.size(0)))
    perm = torch.tensor(perm)
    perm = perm.to(torch.long)
    neg_row, neg_col = neg_row[perm], neg_col[perm]

    neg_adj_mask[neg_row, neg_col] = 0
    data.train_neg_adj_mask = neg_adj_mask

    row, col = neg_row[:n_v], neg_col[:n_v]
    data.val_neg_edge_index = torch.stack([row, col], dim=0)

    row, col = neg_row[n_v:n_v + n_t], neg_col[n_v:n_v + n_t]
    data.test_neg_edge_index = torch.stack([row, col], dim=0)

    return data

In [ ]:
def eval_train_test_split_edges(data):
    r"""Splits the edges of a :obj:`torch_geometric.data.Data` object
    into positive and negative train/val/test edges, and adds attributes of
    `train_pos_edge_index`, `train_neg_adj_mask`, `val_pos_edge_index`,
    `val_neg_edge_index`, `test_pos_edge_index`, and `test_neg_edge_index`
    to :attr:`data`.

    Args:
        data (Data): The data object.
        val_ratio (float, optional): The ratio of positive validation
            edges. (default: :obj:`0.05`)
        test_ratio (float, optional): The ratio of positive test
            edges. (default: :obj:`0.1`)

    :rtype: :class:`torch_geometric.data.Data`
    """

    assert 'batch' not in data  # No batch-mode.

    num_nodes = data.num_nodes
    row, col = data.edge_index
    #data.edge_index = None
    attr = data.edge_attr

    # Return upper triangular portion.
    #mask = row < col
    #row, col = row[mask], col[mask]

    
    # Positive edges.
    perm = torch.randperm(row.size(0))
    row, col = row[perm], col[perm]
    attr=attr[perm]


    r, c = row[:], col[:]
    data.test_pos_edge_index = torch.stack([r, c], dim=0)
    data.test_pos_edge_attr = attr[:]

    # Negative edges.
    neg_adj_mask = torch.ones(num_nodes, num_nodes, dtype=torch.uint8)
    neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
    neg_adj_mask[row, col] = 0

    neg_row, neg_col = neg_adj_mask.nonzero().t()
    perm = random.sample(range(neg_row.size(0)),len(row[:]))
    perm = torch.tensor(perm)
    perm = perm.to(torch.long)
    neg_row, neg_col = neg_row[perm], neg_col[perm]

    row, col = neg_row[:], neg_col[:]
    data.test_neg_edge_index = torch.stack([row, col], dim=0)

    return data

## Learning models

In [ ]:
class Encoder_VGAE(nn.Module):
    def __init__(self, in_channels, out_channels, isClassificationTask=False):
        super(Encoder_VGAE, self).__init__()
        self.isClassificationTask=isClassificationTask
        self.conv_gene_drug = SAGEConv(in_channels, 2*out_channels, )
        self.conv_protein_protein = SAGEConv(in_channels, 2*out_channels, )
        self.conv_drug_phenotype = SAGEConv(in_channels, 2*out_channels, )
        self.conv_gene_phenotype = SAGEConv(in_channels, 2*out_channels, )
        self.conv_drug_pathway = SAGEConv(in_channels, 2*out_channels, )
        self.conv_drug_sim = SAGEConv(in_channels, 2*out_channels, )
        self.conv_AD_gene_pathway = SAGEConv(in_channels, 2*out_channels, )
        self.bn = nn.BatchNorm1d(7*2*out_channels)
        #variational encoder
        self.conv_mu = SAGEConv(7*2*out_channels, out_channels, )
        self.conv_logvar = SAGEConv(7*2*out_channels, out_channels,)

    def forward(self,x,edge_index,edge_attr):
        
        x = F.dropout(x, training=self.training)
        
        index_gene_drug=(edge_attr==0).nonzero().reshape(1,-1)[0]
        edge_index_gene_drug=edge_index[:, index_gene_drug]
        
        index_protein_protein=(edge_attr==1).nonzero().reshape(1,-1)[0]
        edge_index_protein_protein=edge_index[:, index_protein_protein]
        
        index_drug_phenotype=(edge_attr==2).nonzero().reshape(1,-1)[0]
        edge_index_drug_phenotype=edge_index[:, index_drug_phenotype]
        
        index_gene_phenotype=(edge_attr==3).nonzero().reshape(1,-1)[0]
        edge_index_gene_phenotype=edge_index[:, index_gene_phenotype]
        
        index_drug_pathway=(edge_attr==4).nonzero().reshape(1,-1)[0]
        edge_index_drug_pathway=edge_index[:, index_drug_pathway]
        
        index_drug_sim=(edge_attr==5).nonzero().reshape(1,-1)[0]
        edge_index_drug_sim=edge_index[:, index_drug_sim]
        
        index_AD_gene_pathway=(edge_attr==6).nonzero().reshape(1,-1)[0]
        edge_index_AD_gene_pathway=edge_index[:, index_AD_gene_pathway]
        
        x_gene_drug = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_gene_drug)), p=0.5, training=self.training)
        x_protein_protein = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_protein_protein)), p=0.5, training=self.training)
        x_drug_phenotype = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_drug_phenotype)), p=0.5, training=self.training)
        x_gene_phenotype = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_gene_phenotype)), p=0.5, training=self.training)
        x_drug_pathway = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_drug_pathway)), p=0.5, training=self.training)
        x_drug_sim = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_drug_sim)), p=0.5, training=self.training)
        x_AD_gene_pathway = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_AD_gene_pathway)), p=0.5, training=self.training)
        
        
        
        
        
        
        x=self.bn(torch.cat([x_gene_drug,x_protein_protein,x_drug_phenotype,x_gene_phenotype,
                             x_drug_pathway,x_drug_sim ,x_AD_gene_pathway   
        ],dim=1))
        
        return self.conv_mu(x,edge_index), self.conv_logvar(x,edge_index)

In [ ]:
class GCN_auto_node(torch.nn.Module):
    def __init__(self,num_features,hidden_channels,num_class,autoencoder):
        super(GCN_auto_node, self).__init__()
        self.autoencoder = autoencoder
        self.conv1 = SAGEConv(num_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, num_class)

    def forward(self,x, edge_index,edge_attr, node_edge_index ):
        z = self.autoencoder.encode(x, edge_index,edge_attr)
        z_1 = self.conv1(z, node_edge_index)
        z_1 = z_1.relu()
        z_1 = F.dropout(z_1, p=0.5, training=self.training)
        z_1 = self.conv2(z_1, edge_index)
        return z_1,z



In [ ]:
def train(data):
    model.train()
    optimizer.zero_grad()
    class_inf,z=model(data.x, data.train_pos_edge_index,data.train_pos_edge_attr,data.node_train_edge)
    class_inf=class_inf
    node_criteria=nn.BCEWithLogitsLoss(weight=data.node_mask)
    class_loss=node_criteria(class_inf.squeeze(), data.y.float())
    
    loss = model.autoencoder.recon_loss(z, data.train_pos_edge_index)
    
    loss_list= [loss,class_loss]
    
    #loss = loss+class_loss
    #loss.backward()
    #assert len(loss_list) == num_tasks
    optimizer.pc_backward(loss_list)
    optimizer.step()
    
    

def eval_fu(data):
    model.eval()
    with torch.no_grad():
        class_inf,z=model(data.x, data.train_pos_edge_index,data.train_pos_edge_attr,data.node_test_edge)
        class_inf=class_inf.sigmoid()
        edge_auc,edge_prc=model.autoencoder.test(z, data.test_pos_edge_index, data.test_neg_edge_index)
        class_inf=class_inf.squeeze().detach().cpu().numpy()
        y=data.y.squeeze().detach().cpu().numpy()
        class_mask=data.node_mask.squeeze().detach().cpu().numpy()
        class_inf= class_inf[class_mask.nonzero()]
        y=y[class_mask.nonzero()] 
        node_auc=metrics.roc_auc_score(y,class_inf)
    return edge_auc,edge_prc,node_auc

    
def test(data,data_split):
    model.eval()
    with torch.no_grad():
        class_inf,z=model(data.x, data.train_pos_edge_index,data.train_pos_edge_attr,data.node_whole_edge)
        class_inf=class_inf.sigmoid()
        edge_auc,edge_prc=model.autoencoder.test(z, data_split.test_pos_edge_index, data_split.test_neg_edge_index)
        class_inf=class_inf.squeeze().detach().cpu().numpy()
        
        y=data.y.squeeze().detach().cpu().numpy()
        class_mask=data.node_mask.squeeze().detach().cpu().numpy()
        class_inf= class_inf[class_mask.nonzero()]
        y=y[class_mask.nonzero()] 
        node_auc=metrics.roc_auc_score(y,class_inf)
        node_prc=metrics.average_precision_score(y,class_inf)
    return edge_auc,edge_prc,node_auc,node_prc


# basic model only pretrain

In [ ]:
node_feature=torch.tensor(node_feature_np, dtype=torch.float)
edge_index_1=edge_index
edge=torch.tensor(edge_index[['node1', 'node2']].values, dtype=torch.long)
node_train_edge=torch.tensor(train_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)
node_test_edge=torch.tensor(test_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)
node_whole_edge=torch.tensor(all_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)

node_class=torch.tensor(node_class, dtype=torch.long)
node_class_mask=torch.tensor(node_class_mask, dtype=torch.long)
edge_attr=torch.tensor(edge_index['type'].values,dtype=torch.long)
data = Data(x=node_feature,
            edge_index=edge.t().contiguous(),
            edge_attr=edge_attr,
            y=node_class
           )
data.node_train_edge=node_train_edge
data.node_test_edge=node_test_edge
data.node_whole_edge=node_whole_edge
data.node_mask=node_class_mask
data=data.to(device_id)

In [ ]:
data_whole=train_test_split_edges(data)
data_split=eval_train_test_split_edges(data)

model=GCN_auto_node(embedding_size,64,1,VGAE((Encoder_VGAE(node_feature.shape[1], embedding_size)))).to(device_id)
test(data_whole,data_split)

In [ ]:
edge_attr_dict={'gene-drug':0,'protein-protein':1, 'drug-phenotype':2,'gene-phenotype':3,'drug_pathway':4,'drug_sim':5,'AD_gene_pathway':6}#'mutation':3,


In [ ]:
edge_type_list=['gene-drug','protein-protein','drug-phenotype','gene-phenotype','drug_pathway','drug_sim','AD_gene_pathway']
edge_type_ind=[0,1,2,3,4,5,6]
for i in range(len(edge_type_ind)):
    edge_index_1=edge_index[edge_index.type==edge_type_ind[i]]
    edge=torch.tensor(edge_index_1[['node1', 'node2']].values, dtype=torch.long)
    node_class=torch.tensor(node_class, dtype=torch.long)
    edge_attr=torch.tensor(edge_index_1['type'].values,dtype=torch.long)
    data_split = Data(x=node_feature,
                edge_index=edge.t().contiguous(),
                edge_attr=edge_attr,
                y=node_class
               )
    model=GCN_auto_node(embedding_size,64,1,VGAE((Encoder_VGAE(node_feature.shape[1], embedding_size)))).to(device_id)
    data_split=eval_train_test_split_edges(data_split)
    edge_auc,edge_prc,_,_=test(data_whole,data_split)
    print('edge_type :{} edge_auc: {:.4f}, edge_prc: {:.4f}'.format(edge_type_list[i] ,edge_auc,edge_prc))


In [ ]:
del model, data_whole
gc.collect()
del data_split
with torch.cuda.device(device):
        torch.cuda.empty_cache()
    

# model with random inition

In [ ]:
node_feature=nn.Embedding(len(le.classes_), 400)
node_feature=node_feature.weight.data.uniform_(-1, 1)
edge_index_1=edge_index
edge=torch.tensor(edge_index[['node1', 'node2']].values, dtype=torch.long)
node_train_edge=torch.tensor(train_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)
node_test_edge=torch.tensor(test_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)
node_whole_edge=torch.tensor(all_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)

node_class=torch.tensor(node_class, dtype=torch.long)
node_class_mask=torch.tensor(node_class_mask, dtype=torch.long)
edge_attr=torch.tensor(edge_index['type'].values,dtype=torch.long)
data = Data(x=node_feature,
            edge_index=edge.t().contiguous(),
            edge_attr=edge_attr,
            y=node_class
           )
data.node_train_edge=node_train_edge
data.node_test_edge=node_test_edge
data.node_whole_edge=node_whole_edge
data.node_mask=node_class_mask
data=data.to(device_id)

In [ ]:
data_whole=train_test_split_edges(data)
data_split=eval_train_test_split_edges(data)

model=GCN_auto_node(embedding_size,64,1,VGAE((Encoder_VGAE(node_feature.shape[1], embedding_size)))).to(device_id)
learnable_params=model.parameters()
optimizer = PCGrad(torch.optim.Adam(learnable_params)) 
#optimizer=torch.optim.Adam(learnable_params)
for i in range(200):
    train(data_whole)
    edge_auc,edge_prc,node_auc=eval_fu(data_whole)
    print('epoch :{} edge_auc: {:.4f}, edge_prc: {:.4f}, node_auc: {:.4f}'.format(i ,edge_auc,edge_prc,node_auc))
    

In [ ]:
test(data_whole,data_split)

In [ ]:
edge_type_list=['gene-drug','protein-protein','drug-phenotype','gene-phenotype','drug_pathway','drug_sim','AD_gene_pathway']
edge_type_ind=[0,1,2,3,4,5,6]
for i in range(len(edge_type_ind)):
    edge_index_1=edge_index[edge_index.type==edge_type_ind[i]]
    edge=torch.tensor(edge_index_1[['node1', 'node2']].values, dtype=torch.long)
    node_class=torch.tensor(node_class, dtype=torch.long)
    edge_attr=torch.tensor(edge_index_1['type'].values,dtype=torch.long)
    data_split = Data(x=node_feature,
                edge_index=edge.t().contiguous(),
                edge_attr=edge_attr,
                y=node_class
               )
    data_split=eval_train_test_split_edges(data_split)
    edge_auc,edge_prc,_,_=test(data_whole,data_split)
    print('edge_type :{} edge_auc: {:.4f}, edge_prc: {:.4f}'.format(edge_type_list[i] ,edge_auc,edge_prc))
    

In [ ]:
del model, data_whole
gc.collect()
del data_split
with torch.cuda.device(device):
        torch.cuda.empty_cache()
    

# model with pre-train

In [ ]:
node_feature=torch.tensor(node_feature_np, dtype=torch.float)
edge_index_1=edge_index
edge=torch.tensor(edge_index[['node1', 'node2']].values, dtype=torch.long)
node_train_edge=torch.tensor(train_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)
node_test_edge=torch.tensor(test_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)
node_whole_edge=torch.tensor(all_node_edge_index[['node1', 'node2']].values.T, dtype=torch.long)

node_class=torch.tensor(node_class, dtype=torch.long)
node_class_mask=torch.tensor(node_class_mask, dtype=torch.long)
edge_attr=torch.tensor(edge_index['type'].values,dtype=torch.long)
data = Data(x=node_feature,
            edge_index=edge.t().contiguous(),
            edge_attr=edge_attr,
            y=node_class
           )
data.node_train_edge=node_train_edge
data.node_test_edge=node_test_edge
data.node_whole_edge=node_whole_edge
data.node_mask=node_class_mask
data=data.to(device_id)

In [ ]:
data_whole=train_test_split_edges(data)
data_split=eval_train_test_split_edges(data)

model=GCN_auto_node(embedding_size,64,1,VGAE((Encoder_VGAE(node_feature.shape[1], embedding_size)))).to(device_id)
learnable_params=model.parameters()
optimizer = PCGrad(torch.optim.Adam(learnable_params)) 
#optimizer=torch.optim.Adam(learnable_params)
for i in range(200):
    train(data_whole)
    edge_auc,edge_prc,node_auc=eval_fu(data_whole)
    print('epoch :{} edge_auc: {:.4f}, edge_prc: {:.4f}, node_auc: {:.4f}'.format(i ,edge_auc,edge_prc,node_auc))
    

In [ ]:
test(data_whole,data_split)

In [ ]:
edge_type_list=['gene-drug','protein-protein','drug-phenotype','gene-phenotype','drug_pathway','drug_sim','AD_gene_pathway']
edge_type_ind=[0,1,2,3,4,5,6]
for i in range(len(edge_type_ind)):
    edge_index_1=edge_index[edge_index.type==edge_type_ind[i]]
    edge=torch.tensor(edge_index_1[['node1', 'node2']].values, dtype=torch.long)
    node_class=torch.tensor(node_class, dtype=torch.long)
    edge_attr=torch.tensor(edge_index_1['type'].values,dtype=torch.long)
    data_split = Data(x=node_feature,
                edge_index=edge.t().contiguous(),
                edge_attr=edge_attr,
                y=node_class
               )
    data_split=eval_train_test_split_edges(data_split)
    edge_auc,edge_prc,_,_=test(data_whole,data_split)
    print('edge_type :{} edge_auc: {:.4f}, edge_prc: {:.4f}'.format(edge_type_list[i] ,edge_auc,edge_prc))
    

In [ ]:
model.eval()
z=model.autoencoder.encode(data_whole.x, data_whole.edge_index, data_whole.edge_attr)
z_np = z.squeeze().detach().cpu().numpy()
z_np.shape
pickle.dump(z_np, open('./AD_project/pre_process_data/drug_comb_node_embedding_comb_with_pretrain_kh.pkl', 'wb'))
torch.save(model.state_dict(), './AD_project/pre_process_data/drug_comb_VAE_encoders_comb_withour_pretrain_kh.pkl')

In [ ]:
del model, data_whole
gc.collect()
with torch.cuda.device(device):
    torch.cuda.empty_cache()
del data_split
with torch.cuda.device(device):
    torch.cuda.empty_cache()